# Generating Jazz Music using LSTMs
# Data provided by Coursera

In [1]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [2]:
IPython.display.Audio('./data/30s_seq.mp3')

In [3]:
X, Y, n_v, indices = load_music_utils()

In [4]:
print('X shape is ' + str(X.shape))
print('Y shape is ' + str(Y.shape))
print('Length of the one hot vector is ' + str(n_v))
print('The shape of X is of the form (m, Tx, input_length)')
print('The shape of Y is of the form (Ty, m, input_length)')
print('----------------------------------------------------------------------------------------------------------\n')
print('Dictionary for the indice values for the one hot vector is ' + str(indices))
print('----------------------------------------------------------------------------------------------------------\n')

X shape is (60, 30, 78)
Y shape is (30, 60, 78)
Length of the one hot vector is 78
The shape of X is of the form (m, Tx, input_length)
The shape of Y is of the form (Ty, m, input_length)
----------------------------------------------------------------------------------------------------------

Dictionary for the indice values for the one hot vector is {0: 'X,0.250,<M-2,m-6>', 1: 'C,0.250,<d2,A-4>', 2: 'C,0.250,<d4,M-2>', 3: 'C,0.250,<d1,P-5>', 4: 'S,0.333', 5: 'C,0.250', 6: 'C,0.250,<P4,m-2>', 7: 'S,0.250,<P1,d-5>', 8: 'A,0.250,<m2,P-4>', 9: 'A,0.333,<P1,d-5>', 10: 'X,0.250,<d1,P-5>', 11: 'S,0.250,<P5,A1>', 12: 'S,0.667,<m3,m-3>', 13: 'C,0.250,<P1,d-5>', 14: 'S,0.667,<d5,P1>', 15: 'C,0.500,<m2,P-4>', 16: 'S,0.250,<d4,M-2>', 17: 'A,0.250,<P1,d-5>', 18: 'A,0.250,<P-4,d-8>', 19: 'C,0.333,<m7,M3>', 20: 'X,0.250,<d2,A-4>', 21: 'C,0.250,<dd5,d1>', 22: 'C,0.250,<m2,P-4>', 23: 'C,0.667,<d6,m2>', 24: 'C,0.333,<d1,P-5>', 25: 'S,0.500,<m7,M3>', 26: 'S,0.500,<d1,P-5>', 27: 'A,0.250,<P4,m-2>', 28: 

## The above data has X and Y as the training set and labels. Y is just X but one step to the left

In [5]:
n_a = 64 # Number of hidden activations

In [6]:
reshape_operation = Reshape((1, 78))
LSTM_Cell = LSTM(n_a, return_state=True)
probs = Dense(n_v, activation='softmax')

In [15]:
def create_model(T_x, n_a, n_values):
    
    X = Input(shape=(T_x, n_values))
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    
    a = a0
    c = c0
    
    outputs = []
    
    for t in range(T_x):
        
        x = Lambda(lambda x: X[:,t,:])(X)
        x = reshape_operation(x)
        
        a, _, c = LSTM_Cell(x, initial_state=[a, c])
        y = probs(a)
        outputs.append(y)
    
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    return model


In [16]:
model = create_model(T_x = 30, n_a = n_a, n_values=n_v)

In [17]:
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
samples = 60
a0 = np.zeros((samples, n_a))
c0 = np.zeros((samples, n_a))

model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
60/60 [==============================] - 17s 283ms/step - loss: 125.7930 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.0000e+00 - dense_1_accuracy_1: 0.0833 - dense_1_accuracy_2: 0.0667 - dense_1_accuracy_3: 0.0500 - dense_1_accuracy_4: 0.0500 - dense_1_accuracy_5: 0.0500 - dense_1_accuracy_6: 0.1000 - dense_1_accuracy_7: 0.0333 - dense_1_accuracy_8: 0.0833 - dense_1_accuracy_9: 0.0333 - dense_1_accuracy_10: 0.0500 - dense_1_accuracy_11: 0.0667 - dense_1_accuracy_12: 0.0333 - dense_1_accuracy_13: 0.1500 - dense_1_accuracy_14: 0.0667 - dense_1_accuracy_15: 0.0667 - dense_1_accuracy_16: 0.0500 - dense_1_accuracy_17: 0.0167 - dense_1_accuracy_18: 0.0667 - dense_1_accuracy_19: 0.0500 - dense_1_accuracy_20: 0.0333 - dense_1_accuracy_21: 0.0333 - dense_1_accuracy_22: 0.0833 - dense_1_accuracy_23: 0.0667 - dense_1_accuracy_24: 0.0333 - dense_1_accuracy_25: 0.1000 - dense_1_accuracy_26: 0.0667 - dense_1_accuracy_27: 0.0500 - dense_1_accuracy_28: 0.0500 - dense_1_accuracy_29: 0.0

60/60 [==============================] - 0s 2ms/step - loss: 97.1451 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.1000 - dense_1_accuracy_1: 0.1833 - dense_1_accuracy_2: 0.1667 - dense_1_accuracy_3: 0.1500 - dense_1_accuracy_4: 0.2167 - dense_1_accuracy_5: 0.0833 - dense_1_accuracy_6: 0.1333 - dense_1_accuracy_7: 0.1833 - dense_1_accuracy_8: 0.1500 - dense_1_accuracy_9: 0.1333 - dense_1_accuracy_10: 0.1833 - dense_1_accuracy_11: 0.1500 - dense_1_accuracy_12: 0.2000 - dense_1_accuracy_13: 0.2000 - dense_1_accuracy_14: 0.2333 - dense_1_accuracy_15: 0.1833 - dense_1_accuracy_16: 0.2167 - dense_1_accuracy_17: 0.1167 - dense_1_accuracy_18: 0.1833 - dense_1_accuracy_19: 0.1667 - dense_1_accuracy_20: 0.1500 - dense_1_accuracy_21: 0.1833 - dense_1_accuracy_22: 0.1333 - dense_1_accuracy_23: 0.1667 - dense_1_accuracy_24: 0.1167 - dense_1_accuracy_25: 0.3000 - dense_1_accuracy_26: 0.1000 - dense_1_accuracy_27: 0.1833 - dense_1_accuracy_28: 0.1833 - dense_1_accuracy_29: 0.0000e+00
Epoch 10/100

60/60 [==============================] - 0s 2ms/step - loss: 68.8741 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.0667 - dense_1_accuracy_1: 0.2833 - dense_1_accuracy_2: 0.3333 - dense_1_accuracy_3: 0.2500 - dense_1_accuracy_4: 0.3333 - dense_1_accuracy_5: 0.2833 - dense_1_accuracy_6: 0.3000 - dense_1_accuracy_7: 0.3333 - dense_1_accuracy_8: 0.5167 - dense_1_accuracy_9: 0.4667 - dense_1_accuracy_10: 0.3667 - dense_1_accuracy_11: 0.4000 - dense_1_accuracy_12: 0.4833 - dense_1_accuracy_13: 0.4000 - dense_1_accuracy_14: 0.3000 - dense_1_accuracy_15: 0.2833 - dense_1_accuracy_16: 0.4167 - dense_1_accuracy_17: 0.3333 - dense_1_accuracy_18: 0.3333 - dense_1_accuracy_19: 0.4167 - dense_1_accuracy_20: 0.4000 - dense_1_accuracy_21: 0.3333 - dense_1_accuracy_22: 0.5000 - dense_1_accuracy_23: 0.4000 - dense_1_accuracy_24: 0.3333 - dense_1_accuracy_25: 0.4833 - dense_1_accuracy_26: 0.3833 - dense_1_accuracy_27: 0.4667 - dense_1_accuracy_28: 0.3333 - dense_1_accuracy_29: 0.0000e+00
Epoch 18/100

60/60 [==============================] - 0s 1ms/step - loss: 45.1425 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.0667 - dense_1_accuracy_1: 0.4000 - dense_1_accuracy_2: 0.4667 - dense_1_accuracy_3: 0.4167 - dense_1_accuracy_4: 0.5667 - dense_1_accuracy_5: 0.5500 - dense_1_accuracy_6: 0.6000 - dense_1_accuracy_7: 0.7500 - dense_1_accuracy_8: 0.7000 - dense_1_accuracy_9: 0.7667 - dense_1_accuracy_10: 0.7000 - dense_1_accuracy_11: 0.7500 - dense_1_accuracy_12: 0.8000 - dense_1_accuracy_13: 0.7333 - dense_1_accuracy_14: 0.5667 - dense_1_accuracy_15: 0.7333 - dense_1_accuracy_16: 0.7667 - dense_1_accuracy_17: 0.7500 - dense_1_accuracy_18: 0.7333 - dense_1_accuracy_19: 0.7667 - dense_1_accuracy_20: 0.7667 - dense_1_accuracy_21: 0.7667 - dense_1_accuracy_22: 0.7667 - dense_1_accuracy_23: 0.7333 - dense_1_accuracy_24: 0.6833 - dense_1_accuracy_25: 0.7000 - dense_1_accuracy_26: 0.6667 - dense_1_accuracy_27: 0.8167 - dense_1_accuracy_28: 0.7833 - dense_1_accuracy_29: 0.0000e+00
Epoch 26/100

60/60 [==============================] - 0s 2ms/step - loss: 28.4165 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.0667 - dense_1_accuracy_1: 0.4333 - dense_1_accuracy_2: 0.6500 - dense_1_accuracy_3: 0.6333 - dense_1_accuracy_4: 0.7833 - dense_1_accuracy_5: 0.8833 - dense_1_accuracy_6: 0.9333 - dense_1_accuracy_7: 0.9667 - dense_1_accuracy_8: 0.9333 - dense_1_accuracy_9: 1.0000 - dense_1_accuracy_10: 0.9167 - dense_1_accuracy_11: 0.9833 - dense_1_accuracy_12: 1.0000 - dense_1_accuracy_13: 1.0000 - dense_1_accuracy_14: 0.9667 - dense_1_accuracy_15: 0.9833 - dense_1_accuracy_16: 1.0000 - dense_1_accuracy_17: 1.0000 - dense_1_accuracy_18: 0.9667 - dense_1_accuracy_19: 0.9833 - dense_1_accuracy_20: 1.0000 - dense_1_accuracy_21: 0.9667 - dense_1_accuracy_22: 0.9500 - dense_1_accuracy_23: 0.9667 - dense_1_accuracy_24: 0.9833 - dense_1_accuracy_25: 0.9500 - dense_1_accuracy_26: 1.0000 - dense_1_accuracy_27: 0.9667 - dense_1_accuracy_28: 0.9500 - dense_1_accuracy_29: 0.0000e+00
Epoch 34/100

...

60/60 [==============================] - 0s 2ms/step - loss: 9.6735 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.1000 - dense_1_accuracy_1: 0.5833 - dense_1_accuracy_2: 0.8667 - dense_1_accuracy_3: 0.9833 - dense_1_accuracy_4: 0.9833 - dense_1_accuracy_5: 0.9833 - dense_1_accuracy_6: 1.0000 - dense_1_accuracy_7: 1.0000 - dense_1_accuracy_8: 1.0000 - dense_1_accuracy_9: 1.0000 - dense_1_accuracy_10: 1.0000 - dense_1_accuracy_11: 1.0000 - dense_1_accuracy_12: 1.0000 - dense_1_accuracy_13: 1.0000 - dense_1_accuracy_14: 1.0000 - dense_1_accuracy_15: 1.0000 - dense_1_accuracy_16: 1.0000 - dense_1_accuracy_17: 1.0000 - dense_1_accuracy_18: 1.0000 - dense_1_accuracy_19: 1.0000 - dense_1_accuracy_20: 1.0000 - dense_1_accuracy_21: 1.0000 - dense_1_accuracy_22: 1.0000 - dense_1_accuracy_23: 1.0000 - dense_1_accuracy_24: 1.0000 - dense_1_accuracy_25: 1.0000 - dense_1_accuracy_26: 1.0000 - dense_1_accuracy_27: 1.0000 - dense_1_accuracy_28: 1.0000 - dense_1_accuracy_29: 0.0000e+00
Epoch 66/100


60/60 [==============================] - 0s 1ms/step - loss: 8.8340 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.1000 - dense_1_accuracy_1: 0.6000 - dense_1_accuracy_2: 0.8833 - dense_1_accuracy_3: 1.0000 - dense_1_accuracy_4: 0.9833 - dense_1_accuracy_5: 1.0000 - dense_1_accuracy_6: 1.0000 - dense_1_accuracy_7: 1.0000 - dense_1_accuracy_8: 1.0000 - dense_1_accuracy_9: 1.0000 - dense_1_accuracy_10: 1.0000 - dense_1_accuracy_11: 1.0000 - dense_1_accuracy_12: 1.0000 - dense_1_accuracy_13: 1.0000 - dense_1_accuracy_14: 1.0000 - dense_1_accuracy_15: 1.0000 - dense_1_accuracy_16: 1.0000 - dense_1_accuracy_17: 1.0000 - dense_1_accuracy_18: 1.0000 - dense_1_accuracy_19: 1.0000 - dense_1_accuracy_20: 1.0000 - dense_1_accuracy_21: 1.0000 - dense_1_accuracy_22: 1.0000 - dense_1_accuracy_23: 1.0000 - dense_1_accuracy_24: 1.0000 - dense_1_accuracy_25: 1.0000 - dense_1_accuracy_26: 1.0000 - dense_1_accuracy_27: 1.0000 - dense_1_accuracy_28: 1.0000 - dense_1_accuracy_29: 0.0000e+00
Epoch 74/100


60/60 [==============================] - 0s 2ms/step - loss: 8.4158 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.1000 - dense_1_accuracy_1: 0.6500 - dense_1_accuracy_2: 0.8833 - dense_1_accuracy_3: 1.0000 - dense_1_accuracy_4: 1.0000 - dense_1_accuracy_5: 1.0000 - dense_1_accuracy_6: 1.0000 - dense_1_accuracy_7: 1.0000 - dense_1_accuracy_8: 1.0000 - dense_1_accuracy_9: 1.0000 - dense_1_accuracy_10: 1.0000 - dense_1_accuracy_11: 1.0000 - dense_1_accuracy_12: 1.0000 - dense_1_accuracy_13: 1.0000 - dense_1_accuracy_14: 1.0000 - dense_1_accuracy_15: 1.0000 - dense_1_accuracy_16: 1.0000 - dense_1_accuracy_17: 1.0000 - dense_1_accuracy_18: 1.0000 - dense_1_accuracy_19: 1.0000 - dense_1_accuracy_20: 1.0000 - dense_1_accuracy_21: 1.0000 - dense_1_accuracy_22: 1.0000 - dense_1_accuracy_23: 1.0000 - dense_1_accuracy_24: 1.0000 - dense_1_accuracy_25: 1.0000 - dense_1_accuracy_26: 1.0000 - dense_1_accuracy_27: 1.0000 - dense_1_accuracy_28: 1.0000 - dense_1_accuracy_29: 0.0000e+00
Epoch 82/100


60/60 [==============================] - 0s 2ms/step - loss: 8.0523 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.1000 - dense_1_accuracy_1: 0.6667 - dense_1_accuracy_2: 0.8833 - dense_1_accuracy_3: 1.0000 - dense_1_accuracy_4: 1.0000 - dense_1_accuracy_5: 1.0000 - dense_1_accuracy_6: 1.0000 - dense_1_accuracy_7: 1.0000 - dense_1_accuracy_8: 1.0000 - dense_1_accuracy_9: 1.0000 - dense_1_accuracy_10: 1.0000 - dense_1_accuracy_11: 1.0000 - dense_1_accuracy_12: 1.0000 - dense_1_accuracy_13: 1.0000 - dense_1_accuracy_14: 1.0000 - dense_1_accuracy_15: 1.0000 - dense_1_accuracy_16: 1.0000 - dense_1_accuracy_17: 1.0000 - dense_1_accuracy_18: 1.0000 - dense_1_accuracy_19: 1.0000 - dense_1_accuracy_20: 1.0000 - dense_1_accuracy_21: 1.0000 - dense_1_accuracy_22: 1.0000 - dense_1_accuracy_23: 1.0000 - dense_1_accuracy_24: 1.0000 - dense_1_accuracy_25: 1.0000 - dense_1_accuracy_26: 1.0000 - dense_1_accuracy_27: 1.0000 - dense_1_accuracy_28: 1.0000 - dense_1_accuracy_29: 0.0000e+00
Epoch 90/100


60/60 [==============================] - 0s 2ms/step - loss: 7.7706 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.1000 - dense_1_accuracy_1: 0.6833 - dense_1_accuracy_2: 0.8833 - dense_1_accuracy_3: 1.0000 - dense_1_accuracy_4: 1.0000 - dense_1_accuracy_5: 1.0000 - dense_1_accuracy_6: 1.0000 - dense_1_accuracy_7: 1.0000 - dense_1_accuracy_8: 1.0000 - dense_1_accuracy_9: 1.0000 - dense_1_accuracy_10: 1.0000 - dense_1_accuracy_11: 1.0000 - dense_1_accuracy_12: 1.0000 - dense_1_accuracy_13: 1.0000 - dense_1_accuracy_14: 1.0000 - dense_1_accuracy_15: 1.0000 - dense_1_accuracy_16: 1.0000 - dense_1_accuracy_17: 1.0000 - dense_1_accuracy_18: 1.0000 - dense_1_accuracy_19: 1.0000 - dense_1_accuracy_20: 1.0000 - dense_1_accuracy_21: 1.0000 - dense_1_accuracy_22: 1.0000 - dense_1_accuracy_23: 1.0000 - dense_1_accuracy_24: 1.0000 - dense_1_accuracy_25: 1.0000 - dense_1_accuracy_26: 1.0000 - dense_1_accuracy_27: 1.0000 - dense_1_accuracy_28: 1.0000 - dense_1_accuracy_29: 0.0000e+00
Epoch 98/100


##  Predicting New Music from learned model

In [23]:
def music_model_for_prediction(T_y):
    
    x0 = Input(shape=(1, n_v))
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0
    outputs = []
    
    for t in range(T_y):
        
        a, _, c = LSTM_Cell(x, initial_state=[a, c])
        out = probs(a)
        outputs.append(out)
        x = Lambda(one_hot)(out)
    
    model = Model(inputs=[x0, a0, c0], outputs=outputs)
    return model
        

In [26]:
prediction_model = music_model_for_prediction(50)
x_initial = np.zeros((1, 1, 78))
a_initial = np.zeros((1, n_a))
c_initial = np.zeros((1, n_a))

In [28]:
def predict_and_sample(prediction_model=prediction_model):
    predictions = prediction_model.predict([x_initial, a_initial, c_initial])
    indices = np.argmax(predictions, axis=-1)
    results = to_categorical(indices, num_classes=78)
    return results, indices

results, indices = predict_and_sample()
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

np.argmax(results[12]) = 28
np.argmax(results[17]) = 21
list(indices[12:18]) = [array([28]), array([21]), array([67]), array([6]), array([28]), array([21])]


In [30]:
out_stream = generate_music(prediction_model)

Predicting new values for different set of chords.
Generated 51 sounds using the predicted values for the set of chords ("1") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("2") and after pruning
Generated 50 sounds using the predicted values for the set of chords ("3") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("4") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi


In [31]:
IPython.display.Audio('./output/my_music.mp3')